<a href="https://colab.research.google.com/github/ruilejin/LLM-review-sentiment/blob/main/classifier_roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from torch.utils.data import DataLoader, Dataset
import transformers
from transformers import AdamW
from transformers import RobertaTokenizerFast
from transformers import TrainingArguments, Trainer
from transformers import RobertaForSequenceClassification
from sklearn.model_selection import KFold

Import data

In [ ]:
X_train_drama_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/X_train_drama.csv',sep=',')
y_train_drama_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/y_train_drama.csv',sep=',')
X_test_drama_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/X_test_drama.csv',sep=',')
y_test_drama_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/y_test_drama.csv',sep=',')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
X_train_comedy_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/X_train_comedy.csv',sep=',')
y_train_comedy_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/y_train_comedy.csv',sep=',')
X_test_comedy_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/X_test_comedy.csv',sep=',')
y_test_comedy_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/y_test_comedy.csv',sep=',')

In [ ]:
X_train_documentary_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/X_train_documentary.csv',sep=',')
y_train_documentary_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/y_train_documentary.csv',sep=',')
X_test_documentary_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/X_test_documentary.csv',sep=',')
y_test_documentary_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/y_test_documentary.csv',sep=',')

In [ ]:
X_train_horror_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/X_train_horror.csv',sep=',')
y_train_horror_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/y_train_horror.csv',sep=',')
X_test_horror_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/X_test_horror.csv',sep=',')
y_test_horror_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/y_test_horror.csv',sep=',')
X_train_horror_df=X_train_horror_df.dropna(axis=1)
y_train_horror_df=y_train_horror_df.dropna(axis=1)
X_test_horror_df=X_test_horror_df.dropna(axis=1)
y_test_horror_df=y_test_horror_df.dropna(axis=1)

In [ ]:
X_train_mystery_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/X_train_mystery.csv',sep=',')
y_train_mystery_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/y_train_mystery.csv',sep=',')
X_test_mystery_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/X_test_mystery.csv',sep=',')
y_test_mystery_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/y_test_mystery.csv',sep=',')

In [ ]:
X_train_action_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/X_train_action.csv',sep=',')
y_train_action_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/y_train_action.csv',sep=',')
X_test_action_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/X_test_action.csv',sep=',')
y_test_action_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/y_test_action.csv',sep=',')
X_train_action_df=X_train_action_df.dropna(axis=1)
y_train_action_df=y_train_action_df.dropna(axis=1)
X_test_action_df=X_test_action_df.dropna(axis=1)
y_test_action_df=y_test_action_df.dropna(axis=1)

In [ ]:
X_train_all_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/all/X_train_all.csv',sep=',')
y_train_all_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/all/y_train_all.csv',sep=',')
X_test_all_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/all/X_test_all.csv',sep=',')
y_test_all_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/all/y_test_all.csv',sep=',')
X_train_all_df=X_train_all_df.dropna(axis=1)
y_train_all_df=y_train_all_df.dropna(axis=1)
X_test_all_df=X_test_all_df.dropna(axis=1)
y_test_all_df=y_test_all_df.dropna(axis=1)

In [ ]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self,encodings,labels=None):
    self.encodings = encodings
    self.labels = labels
  def __getitem__(self,idx):
    item = {key:torch.tensor(val[idx]) for key,val in self.encodings.items()} # We get the dictionary key and that particular example in the tensor representation
    if self.labels:
      item["labels"] = torch.tensor(self.labels[idx]) # We also take a label at particular index
    return item # return item which is a dictionary

  def __len__(self):
    return len(self.encodings["input_ids"])

In [ ]:
def encoding_datasets(tokenizer,X_train, y_train, X_test, y_test):

    tokenizer = tokenizer

    train_encodings = tokenizer(X_train,max_length=64,truncation=True,padding=True,return_tensors='pt')
    test_encodings = tokenizer(X_test,max_length=64,truncation=True,padding=True,return_tensors='pt')

    train_dataset = Dataset(train_encodings,y_train)
    test_dataset = Dataset(test_encodings,y_test)

    return train_dataset,test_dataset

In [ ]:
def prepare_kfold_datasets(tokenizer,k,X_train_df,y_train_df,X_test_df,y_test_df):
    train_datasets=[]
    test_datasets=[]

    for i in range(k):

        X_train = list(X_train_df.iloc[i])
        y_train = list(y_train_df.iloc[i])
        X_test = list(X_test_df.iloc[i])
        y_test = list(y_test_df.iloc[i])

        train_dataset,test_dataset = encoding_datasets(tokenizer,X_train,y_train,X_test,y_test)

        train_datasets.append(train_dataset)
        test_datasets.append(test_dataset)

    return train_datasets,test_datasets

In [ ]:
def compute_metrics(p):
  print(type(p))
  pred,labels = p # transformer output has two thing, label and prediction
  pred = np.argmax(pred,axis=1)

  accuracy = accuracy_score(y_true=labels,y_pred=pred)
  recall = recall_score(y_true=labels,y_pred=pred)
  precision = precision_score(y_true=labels,y_pred=pred)
  f1 = f1_score(y_true=labels,y_pred=pred)

  return{"accuracy":accuracy,"precision":precision,"recall":recall,"f1":f1}

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
roberta_base = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
#roberta_rottentomatoes = RobertaForSequenceClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/roberta-rottentomatoes", num_labels=2)

In [ ]:
train_datasets_drama,test_datasets_drama = prepare_kfold_datasets(tokenizer,5,X_train_drama_df,y_train_drama_df,X_test_drama_df,y_test_drama_df)

In [ ]:
train_datasets_comedy,test_datasets_comedy = prepare_kfold_datasets(tokenizer,5,X_train_comedy_df,y_train_comedy_df,X_test_comedy_df,y_test_comedy_df)

In [ ]:
train_datasets_documentary,test_datasets_documentary = prepare_kfold_datasets(tokenizer,5,X_train_documentary_df,y_train_documentary_df,X_test_documentary_df,y_test_documentary_df)

In [ ]:
train_datasets_horror,test_datasets_horror = prepare_kfold_datasets(tokenizer,5,X_train_horror_df,y_train_horror_df,X_test_horror_df,y_test_horror_df)

In [ ]:
train_datasets_mystery,test_datasets_mystery = prepare_kfold_datasets(tokenizer,5,X_train_mystery_df,y_train_mystery_df,X_test_mystery_df,y_test_mystery_df)

In [ ]:
train_datasets_action,test_datasets_action = prepare_kfold_datasets(tokenizer,5,X_train_action_df,y_train_action_df,X_test_action_df,y_test_action_df)

In [ ]:
train_datasets_all,test_datasets_all = prepare_kfold_datasets(tokenizer,5,X_train_all_df,y_train_all_df,X_test_all_df,y_test_all_df)

By Genres

In [ ]:
# Define Trainer
args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Colab Notebooks/outputs/output_roberta_base_bygenre",
    overwrite_output_dir=False,
    num_train_epochs=2,
    per_device_train_batch_size=8,
    save_total_limit = 2,
    learning_rate=3e-5,
    save_strategy='no',
    load_best_model_at_end = True,
    metric_for_best_model = 'f1',
    fp16 = True,
    logging_steps = 100
)
trainer2 = Trainer(
    model=roberta_base,
    args=args,
    train_dataset=train_datasets_documentary[4],
    eval_dataset=test_datasets_documentary[4],
    compute_metrics=compute_metrics
)


In [ ]:
trainer2.train()

In [ ]:
trainer2.evaluate()

In [ ]:
trainer2.save_model("/content/drive/MyDrive/Colab Notebooks/models/Base_documentary_4")

By all

In [ ]:
# Define Trainer
args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Colab Notebooks/outputs/output_roberta_base_byall",
    overwrite_output_dir=False,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    save_total_limit = 2,
    learning_rate=3e-5,
    save_strategy='no',
    load_best_model_at_end = True,
    metric_for_best_model = 'f1',
    fp16 = True,
    logging_steps = 100
)
trainer3 = Trainer(
    model=roberta_base,
    args=args,
    train_dataset=train_datasets_all[4],
    eval_dataset=test_datasets_all[4],
    compute_metrics=compute_metrics
)


In [ ]:
trainer3.train()

In [ ]:
trainer3.evaluate()

In [ ]:
trainer3.save_model("/content/drive/MyDrive/Colab Notebooks/models/Base_all/Finet_all_4")

In [ ]:
# evaluate on different genre
args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Colab Notebooks/outputs/output_roberta_base_byall",
    overwrite_output_dir=False,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    save_total_limit = 2,
    learning_rate=3e-5,
    save_strategy='no',
    load_best_model_at_end = True,
    metric_for_best_model = 'f1',
    fp16 = True,
    logging_steps = 100
)
trainer4 = Trainer(
    model=RobertaForSequenceClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/models/all/Finet_all_4", num_labels=2),
    args=args,
    train_dataset=train_datasets_action[0],
    eval_dataset=test_datasets_action[4],
    compute_metrics=compute_metrics
)

In [ ]:
trainer4.evaluate()

Ensemble

In [ ]:
def EnsembleClassifier(train_dataset,eval_dataset,cate,model="Base"):
    model=f"./models/{model}_{cate}_0"
    categories=["drama", "comedy", "documentary", "horror", "mystery", "action"]
    if cate in categories:

        args = TrainingArguments(
        output_dir="/content/drive/MyDrive/Colab Notebooks/outputs/output_roberta_base_byall",
        overwrite_output_dir=False,
        num_train_epochs=1,
        per_device_train_batch_size=8,
        save_total_limit = 2,
        learning_rate=3e-5,
        save_strategy='no',
        load_best_model_at_end = True,
        metric_for_best_model = 'f1',
        fp16 = True,
        logging_steps = 100
    )

        trainer = Trainer(
        model=RobertaForSequenceClassification.from_pretrained(model, num_labels=2),
        args=args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics
    )
        print(trainer.evaluate())

In [ ]:
EnsembleClassifier(train_datasets_drama[0],test_datasets_drama[0],"drama")